In [1]:
from lab3_tools import *

# 3) Data

In [2]:
# TODO: Get the tidigits dataset from AFS

In [3]:
path2info('tidigits/disc_4.1.1/tidigits/train/man/ae/z9z6531a.wav')

('man', 'ae', 'z9z6531', 'a')

In [4]:
loadAudio('tidigits/disc_4.1.1/tidigits/train/man/ae/z9z6531a.wav')

LibsndfileError: Error opening 'tidigits/disc_4.1.1/tidigits/train/man/ae/z9z6531a.wav': System error.